In [1]:
from datasets import load_dataset

In [2]:
data = load_dataset("thesven/gsm8k-reasoning")

In [3]:
df = data['train'].to_pandas()

In [4]:
df.head()

,system_prompt,question,answer,short_answer,generation,messages
0,\nYou are an AI assistant that uses a Chain of...,A craft store makes a third of its sales in th...,"Thus, there were 36 - 12 - 9 = <<36-12-9=15>>1...",15,<thinking>\nTo find the number of sales in the...,[{'content': ' You are an AI assistant that us...
1,\nYou are an AI assistant that uses a Chain of...,Marcy is a makeup artist and has agreed to do ...,So she must be applying makeup to 12 tubes of ...,36,"‹thinking›\n\nTo solve this problem, we need t...",[{'content': ' You are an AI assistant that us...
2,\nYou are an AI assistant that uses a Chain of...,Ellie has found an old bicycle in a field and ...,"To fix the rest of the bike as well, she needs...",25,"<thinking>\nTo solve this problem, I need to c...",[{'content': ' You are an AI assistant that us...
3,\nYou are an AI assistant that uses a Chain of...,Janice can type 6 sentences per minute. Today ...,"Thus, she had X = 536 - 278 = <<536-278=258>>2...",258,<thinking>\nTo find out how many sentences Jan...,[{'content': ' You are an AI assistant that us...
4,\nYou are an AI assistant that uses a Chain of...,"During one day, there are 4 boat trips through...",During two days the boat can transport a total...,96,<thinking>\nTo find out how many people the bo...,[{'content': ' You are an AI assistant that us...


In [5]:
print(df["question"][2])

Ellie has found an old bicycle in a field and thinks it just needs some oil to work well again.  She needs 10ml of oil to fix each wheel and will need another 5ml of oil to fix the rest of the bike. How much oil does she need in total to fix the bike?


In [6]:
print(df["answer"][2])

To fix the rest of the bike as well, she needs 20 + 5 = <<20+5=25>>25ml of oil.


In [7]:
print(df["generation"][2])

<thinking>
To solve this problem, I need to calculate the total amount of oil Ellie needs to fix the bike. I'll start by calculating the amount of oil needed for the wheels. She needs 10ml of oil to fix each wheel, and there are 2 wheels on a bike. So, the total amount of oil needed for the wheels is 10ml * 2 = 20ml. Additionally, she needs 5ml of oil to fix the rest of the bike. I can add the amount of oil needed for the wheels and the rest of the bike to get the total amount of oil needed.

</thinking>

<reasoning>
1. Calculate the total amount of oil needed for the wheels: 10ml * 2 = 20ml.
2. Add the amount of oil needed for the rest of the bike: 20ml + 5ml = 25ml.
3. The total amount of oil needed is the sum of the oil needed for the wheels and the rest of the bike.

</reasoning>

‹reflection>
My reasoning seems sound, but I should double-check my math to ensure I didn't make any calculation errors. Upon reviewing my steps, I'm confident that my calculations are correct.

</reflect

In [8]:
print(df["messages"][2])

[{'content': '\nYou are an AI assistant that uses a Chain of Thought (CoT) approach with reflection to answer queries.\n\nFollow these steps:\n- Think through the problem step by step within the ‹thinking> tags.\n- Reflect on your thinking\n\nto check for any errors or improvements within the ‹reflection› tags.\n- Make any necessary adjustments based on your reflection.\n- Provide your final, concise answer within the ‹output> tags.\n\nImportant: The <thinking> and ‹reflection› sections are for your internal reasoning process only.\nDo not include any part of the final answer in these sections.\nThe actual response to the query must be entirely contained within the ‹output› tags.\n\nUse the following format for your response:\n<thinking>\n[Your initial thought process goes here]\n</thinking›\n\n<reasoning>\n[Your step-by-step reasoning goes here. This is your internal thought process, not the final answer. You can create as many reasoning steps as necessary in your process.]\n</reasoni

In [9]:
import re

def process_to_match_r1(gen: str | None):
    if not gen:
        return None

    s = gen

    # Normalize odd quotes/backticks
    s = (s.replace("‹", "<").replace("›", ">")
           .replace("\u2039", "<").replace("\u203A", ">")
           .replace("`", ""))

    # Remove non-required sections entirely
    s = re.sub(r"<(reasoning|reflection|adjustment)>.*?</\1>", "", s, flags=re.DOTALL|re.IGNORECASE)

    # Take the last non-empty thinking/think block (if any)
    think_blocks = re.findall(r"<(thinking|think)>\s*(.*?)\s*</(thinking|think)>",
                             s, flags=re.DOTALL|re.IGNORECASE)
    think_text = ""
    for _, block, _ in think_blocks:
        if block.strip():
            think_text = block.strip()

    # Take the last non-empty output/answer block; required
    ans_blocks = re.findall(r"<(output|answer)>\s*(.*?)\s*</(output|answer)>",
                            s, flags=re.DOTALL|re.IGNORECASE)
    if not ans_blocks:
        return None
    answer_text = ""
    for _, block, _ in ans_blocks:
        if block.strip():
            answer_text = block.strip()

    # In case tags leaked into the answer, strip any remaining tags there
    answer_text = re.sub(r"<[^>]+>", "", answer_text).strip()

    return f"{think_text}</think> <answer>{answer_text}</answer>"

In [10]:
df["response"] = df["generation"].apply(process_to_match_r1)

In [11]:
with open("../integrl/prompts/r1_zero.prompt") as f:
    prompt_template = f.read()

In [12]:
df["prompt"] = df["question"].apply(lambda q: prompt_template.format(question=q))

In [13]:
df.shape

(6914, 8)

In [14]:
df[df.question.str.contains("Janet")].response.iloc[0]

In [15]:
df.shape

(6914, 8)

In [16]:
df = df.dropna(subset=["response"]).reset_index(drop=True)

In [17]:
df.head()

,system_prompt,question,answer,short_answer,generation,messages,response,prompt
0,\nYou are an AI assistant that uses a Chain of...,A craft store makes a third of its sales in th...,"Thus, there were 36 - 12 - 9 = <<36-12-9=15>>1...",15,<thinking>\nTo find the number of sales in the...,[{'content': ' You are an AI assistant that us...,To find the number of sales in the stationery ...,A conversation between User and Assistant. The...
1,\nYou are an AI assistant that uses a Chain of...,"During one day, there are 4 boat trips through...",During two days the boat can transport a total...,96,<thinking>\nTo find out how many people the bo...,[{'content': ' You are an AI assistant that us...,To find out how many people the boat can trans...,A conversation between User and Assistant. The...
2,\nYou are an AI assistant that uses a Chain of...,Paddington has 40 more goats than Washington. ...,"In total, they have 140+180 = <<140+180=320>>3...",320,<thinking>\nI need to find the total number of...,[{'content': ' You are an AI assistant that us...,I need to find the total number of goats that ...,A conversation between User and Assistant. The...
3,\nYou are an AI assistant that uses a Chain of...,John buys 10 packs of magic cards. Each pack ...,So he got 10*5 = <<10*5=50>>50 uncommons,50,"<thinking>\nTo solve this problem, I need to c...",[{'content': ' You are an AI assistant that us...,"To solve this problem, I need to calculate the...",A conversation between User and Assistant. The...
4,\nYou are an AI assistant that uses a Chain of...,Henry took 9 pills a day for 14 days. Of these...,Henry spent a total of 35+6 = <<35+6=41>>41 do...,41,"<thinking>\nTo solve this problem, I need to f...",[{'content': ' You are an AI assistant that us...,"To solve this problem, I need to first calcula...",A conversation between User and Assistant. The...


In [18]:
# print(df[df["response"].str.contains("<thinking>")]["generation"].iloc[0])

In [19]:
# print(df[~df["response"].str.contains(r"</think> <answer>.*</answer>", regex=True)]['generation'].iloc[0])

In [20]:
assert not df["response"].str.contains("<thinking>").any()
assert df["response"].str.contains(r"(?s)</think>\s*<answer>[\s\S]*?</answer>").all()
assert df["prompt"].str.endswith("Assistant: <think>").all()

In [22]:
df[["prompt", "response"]].to_json("../integrl/data/gsm8k/sft.jsonl", orient="records", lines=True)